In [3]:
#
# Near-symmetric distribution-free confidence interval for a quantile `q`.
# Returns indexes into the order statistics.
#
quantile.CI <- function(n, q, alpha=0.05) {
  #
  # Search over a small range of upper and lower order statistics for the 
  # closest coverage to 1-alpha (but not less than it, if possible).
  #
  u <- qbinom(1-alpha/2, n, q) + (-2:2) + 1
  l <- qbinom(alpha/2, n, q) + (-2:2)
  u[u > n] <- Inf
  l[l < 0] <- -Inf
  coverage <- outer(l, u, function(a,b) pbinom(b-1,n,q) - pbinom(a-1,n,q))
  if (max(coverage) < 1-alpha) i <- which(coverage==max(coverage)) else
    i <- which(coverage == min(coverage[coverage >= 1-alpha]))
  i <- i[1]
  #
  # Return the order statistics and the actual coverage.
  #
  u <- rep(u, each=5)[i]
  l <- rep(l, 5)[i]
  return(list(Interval=c(l,u), Coverage=coverage[i]))
}
#
# Example: test coverage via simulation.
#
n <- 100      # Sample size
q <- 0.90     # Percentile
#
# You only have to compute the order statistics once for any given (n,q).
#
lu <- quantile.CI(n, q)$Interval
#
# Generate many random samples from a known distribution and compute 
# CIs from those samples.
#
set.seed(17)
n.sim <- 1e4
index <- function(x, i) ifelse(i==Inf, Inf, ifelse(i==-Inf, -Inf, x[i]))
sim <- replicate(n.sim, index(sort(rnorm(n)), lu))
#
# Compute the proportion of those intervals that cover the percentile.
#
F.q <- qnorm(q)
covers <- sim[1, ] <= F.q & F.q <= sim[2, ]
#
# Report the result.
#
message("Simulation mean coverage was ", signif(mean(covers), 4), 
        "; expected coverage is ", signif(quantile.CI(n,q)$Coverage, 4))

Simulation mean coverage was 0.9503; expected coverage is 0.9523



# from: https://stats.stackexchange.com/questions/99829/how-to-obtain-a-confidence-interval-for-a-percentile
This question, which covers a common situation, deserves a simple, non-approximate answer. Fortunately, there is one.

Suppose X1,…,Xn
are independent values from an unknown distribution F whose qth quantile I will write F−1(q). This means each Xi has a chance of (at least) q of being less than or equal to F−1(q). Consequently the number of Xi less than or equal to F−1(q) has a Binomial(n,q)

distribution.

Motivated by this simple consideration, Gerald Hahn and William Meeker in their handbook Statistical Intervals (Wiley 1991) write

    A two-sided distribution-free conservative 100(1−α)%

confidence interval for F−1(q) is obtained ... as [X(l),X(u)]

where X(1)≤X(2)≤⋯≤X(n)

are the order statistics of the sample. They proceed to say

    One can choose integers 0≤l≤u≤n

symmetrically (or nearly symmetrically) around q(n+1) and as close together as possible subject to the requirements that
B(u−1;n,q)−B(l−1;n,q)≥1−α.(1)

The expression at the left is the chance that a Binomial(n,q)
variable has one of the values {l,l+1,…,u−1}. Evidently, this is the chance that the number of data values Xi falling within the lower 100q% of the distribution is neither too small (less than l) nor too large (u

or greater).

Hahn and Meeker follow with some useful remarks, which I will quote.

    The preceding interval is conservative because the actual confidence level, given by the left-hand side of Equation (1)

, is greater than the specified value 1−α

. ...

It is sometimes impossible to construct a distribution-free statistical interval that has at least the desired confidence level. This problem is particularly acute when estimating percentiles in the tail of a distribution from a small sample. ... In some cases, the analyst can cope with this problem by choosing l
and u

    nonsymmetrically. Another alternative may be to use a reduced confidence level.

Let's work through an example (also provided by Hahn & Meeker). They supply an ordered set of n=100
"measurements of a compound from a chemical process" and ask for a 100(1−α)=95% confidence interval for the q=0.90 percentile. They claim l=85 and u=97 will work.